In [3]:
from tqdm.notebook import trange, tqdm
import requests

In [13]:
#!sed '1,10d' cod.csv > cod_new.csv
import pandas as pd
data = pd.read_csv("cod_new.csv")

/var/folders/gl/2h8nn4ps031g1jysgll17wz00000gn/T/ipykernel_94916/2136347257.py:3: DtypeWarning: Columns (23,24,25,30,31,32,38,44,45,46,48,49,51,52,67,72) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("cod_new.csv")


In [63]:
!wget http://www.crystallography.net/cod/result.php?format=csv -o cod.csv

zsh:1: no matches found: http://www.crystallography.net/cod/result.php?format=csv


In [5]:
!pip install scidownl PyPDF2 transformers torch sentencepiece

In [6]:
import PyPDF2

In [ ]:
!pip install sentencepiece

In [7]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

In [122]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, pipeline

import torch

src_text = [
    """A family of stable and otherwise selectively unachievable 2,6-bisimino-4-R-1,4-dihydropyridinate aluminium (III) dialkyl complexes [AlR'2(4-R-iPrBIPH)] (R = Bn, Allyl; R′ = Me, Et, iBu) have been synthesized, taking advantage of a method for the preparation of the corresponding 4-R-1,4-dihydropiridine precursors developed in our group. All the dihydropyrdinate(−1) dialkyl aluminium complexes have been fully characterized by 1H- 13C-NMR, elemental analysis and in the case 2′a, also by X-ray diffraction studies. Upon heating in toluene solution at 110 °C, the dimethyl derivatives 2a and 2′a dimerize selectively through a double cycloaddition. This reaction leads to the formation of two new C–C bonds that involve the both meta positions of the two 4-R-1,4-dihydropyridinate fragments, resulting the binuclear aluminium species [Me2Al(4-R-iPrHBIP)]2 (R = Bn (3a); allyl (3′a)). Experimental kinetics showed that the dimerization of 2′a obeys second order rate with negative activation entropy, which is consistent with a bimolecular rate-determining step. Controlled methanolysis of both 3a and 3′a release the metal-free dimeric bases, (4-Bn-iPrHBIPH)2 and (4-allyl-iPrHBIPH)2, providing a convenient route to these potentially useful ditopic ligands. When the R′ groups are bulkier than Me (2b, 2′b and 2′c), the dimerization is hindered or fully disabled, favoring the formation of paramagnetic NMR-silent species, which have been identified on the basis of a controlled methanolysis of the final organometallic products. Thus, when a toluene solution of [AlEt2(4-Bn-iPrBIPH)] (2b) was heated at 110 °C, followed by the addition of methanol in excess, it yields a mixture of the dimer (4-Bn-iPrHBIPH)2 and the aromatized base 4-Bn-iPrBIP, in ca. 1 : 2 ratio, indicating that the dimerization of 2b competes with its spontaneous dehydrogenation, yielding a paramagnetic complex containing a AlEt2 unit and a non-innocent (4-Bn-iPrBIP)˙− radical-anion ligand. Similar NMR monitoring experiments on the thermal behavior of [AlEt2(4-allyl-iPrBIPH)] (2′b) and [AliBu2(4-allyl-iPrBIPH)] (2′c) showed that these complexes do not dimerize, but afford exclusively NMR silent products. When such thermally treated samples were subjected to methanolysis, they resulted in mixtures of the alkylated 4-allyl-iPrBIP and non-alkylated iPrBIP ligand, suggesting that dehydrogenation and deallylation reactions take place competitively."""
]

device = "mps"
tokenizer_news = PegasusTokenizer.from_pretrained("google/pegasus-cnn_dailymail", max_position_embeddings=2048)
#tokenizer_xl = PegasusTokenizer.from_pretrained("google/pegasus-large", max_position_embeddings=2048)
model_news = PegasusForConditionalGeneration.from_pretrained("google/pegasus-cnn_dailymail", max_position_embeddings=2048).to(device)
#model_xl = PegasusForConditionalGeneration.from_pretrained("google/pegasus-large", max_position_embeddings=2048).to(device)
batch = tokenizer_news(src_text, truncation=True, padding="longest", return_tensors="pt").to(device)
translated = model_news.generate(**batch)
tgt_text = tokenizer_news.batch_decode(translated, skip_special_tokens=True)
print(tgt_text)

/usr/local/anaconda3/envs/ml/lib/python3.9/site-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 128 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['The dimethyl derivatives 2a and 2′a dimerize selectively through a double cycloaddition.<n>Controlled methanolysis of both 3a and 3′a release the metal-free dimeric bases, (4-Bn-iPrIPH)2 and (4-allyl-iPrPH)2, providing a convenient route to these potentially useful ditopics.']


In [55]:
import requests
from bs4 import BeautifulSoup
import re
import io
import PyPDF2
headers = {'User-Agent': 'Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'}

def download_read_pdf(uri):
    response = requests.get(uri)
    mem = io.BytesIO(response.content)

    pdfReader = PyPDF2.PdfReader(mem)
    
    text = pdfReader.pages[0].extract_text()

    mem.close()
    return text

def get_doi_text(doi):
    try:
        r = requests.get("https://doi.org/"+ doi, headers=headers)
    except:
        raise ValueError("DOI is being weird (can't fetch that)")
    soup = BeautifulSoup(r.content, 'html.parser')
    print(soup)
    abstr = soup.find('meta',attrs={'name':'DCTERMS.abstract'})
    if not abstr:
        abstr = soup.find('meta',attrs={'name':'citation_abstract'})
    if abstr:
        return abstr["content"]
    else:
        abstr = soup(text=re.compile(r'(.*)abstract(.*)', re.I))
        if len(abstr) == 0:
            r = requests.get("http://library.lol/scimag/"+ doi, headers=headers)
            soup = BeautifulSoup(r.content, 'html.parser')
            url = soup.find("div", {"id": "download"}).findNext("a")["href"]
            return download_read_pdf(url)
            
        return abstr[0].parent.findNext('p').get_text()
        

In [56]:
get_doi_text("10.1006/jssc.1995.1407")


<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "https://www.w3.org/TR/html4/loose.dtd">

<html>
<head>
<meta charset="utf-8"/>
<meta content="text/html" http-equiv="Content-Type"/>
<meta content="2; url='/retrieve/articleSelectSinglePerm?Redirect=https%3A%2F%2Fwww.sciencedirect.com%2Fscience%2Farticle%2Fpii%2FS0022459685714077%3Fvia%253Dihub&amp;key=74d0b72207ca2273a32b6eeb26fad96102fb1445'" http-equiv="REFRESH">
<script type="text/javascript">(window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={licenseKey:"ee15051b1b",applicationID:"81847441"};;(()=>{var __webpack_modules__={507:(__unused_webpack_module,__webpack_exports__,__webpack_require__)=>{"use strict";function detectPolyfillFeatures(){const featureStatus={};return checkAndAddFeature("Promise","PROMISE"),checkAndAddFeature("Array.prototype.includes","ARRAY_INCLUDES"),checkAndAddFeature("Object.assign","OBJECT_ASSIGN"),checkAndAddFeature("Object.entries"

''

In [49]:
import re
"""
r = requests.get("http://library.lol/scimag/"+"10.1021/acs.macromol.0c01461", headers={"User-Agent":"Mozilla/5.0"})
soup = BeautifulSoup(r.content, 'html.parser')
url = soup.find("div", {"id": "download"}).findNext("a")["href"]
"""

'\nr = requests.get("http://library.lol/scimag/"+"10.1021/acs.macromol.0c01461", headers={"User-Agent":"Mozilla/5.0"})\nsoup = BeautifulSoup(r.content, \'html.parser\')\nurl = soup.find("div", {"id": "download"}).findNext("a")["href"]\n'

In [48]:
"""
r = requests.get("https://doi.org/"+ "10.1524/zkri.1985.172.1-2.129", headers={"User-Agent":"Mozilla/5.0"})
soup = BeautifulSoup(r.content, 'html.parser')
abstr = soup.find('meta',attrs={'name':'DCTERMS.abstract'})
if not abstr:
    abstr = soup.find('meta',attrs={'name':'og:description'})
if not abstr:
    abstr = soup.find('meta',attrs={'name':'description'})
if not abstr:
    abstr = soup.find('meta',attrs={'name':'Description'})
if abstr:
    print(abstr["content"])
"""

'\nr = requests.get("https://doi.org/"+ "10.1524/zkri.1985.172.1-2.129", headers={"User-Agent":"Mozilla/5.0"})\nsoup = BeautifulSoup(r.content, \'html.parser\')\nabstr = soup.find(\'meta\',attrs={\'name\':\'DCTERMS.abstract\'})\nif not abstr:\n    abstr = soup.find(\'meta\',attrs={\'name\':\'og:description\'})\nif not abstr:\n    abstr = soup.find(\'meta\',attrs={\'name\':\'description\'})\nif not abstr:\n    abstr = soup.find(\'meta\',attrs={\'name\':\'Description\'})\nif abstr:\n    print(abstr["content"])\n'

In [47]:
import os
def get_summary(doi):
    text = get_doi_text(doi)
    inputs = tokenizer_news(text, return_tensors="pt")["input_ids"]
    if inputs.shape[1] > 1024:
        inputs = inputs.narrow(1,0,1024)
    return tokenizer_news.decode(model_news.generate(inputs, max_new_tokens=result_length)[0])

In [11]:
import json

doi_sum = {}
for i,r in tqdm(data.iterrows(), total=data.shape[0]):
    if i % 10 == 0:
        with open('summaries_inorganic.json', 'w') as fp:
            json.dump(doi_sum, fp)
    summ = get_summary(r.doi)
    if summ == "":
        pass
    doi_sum[r.doi] = summ
    if i == 3:
        break

with open('summaries_inorganic.json', 'w') as fp:
    json.dump(doi_sum, fp)

  0%|          | 0/44334 [00:00<?, ?it/s]

NameError: name 'get_summary' is not defined